# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv
data = "../output_data/weather_df.csv" 
cities_df = pd.read_csv(data)
cities_df.head()

,Unnamed: 0,City,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Lat,Lng
0,0,andros town,87.80,66,40,6.93,BS,1592927315,24.70,-77.77
1,1,almaznyy,87.80,40,20,8.95,RU,1592927315,48.04,40.05
2,2,east london,72.99,37,0,4.47,ZA,1592926883,-33.02,27.91
3,3,mataura,39.00,83,8,3.00,NZ,1592926880,-46.19,168.86
4,4,punta arenas,39.09,75,52,21.92,CL,1592926878,-53.15,-70.92


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Create a DataFrame and convert values to float
locations = cities_df[["Lat", "Lng"]].astype(float)

# Store humidity data in a variable called humidity
humidity = cities_df["Humidity"]
max_humidity = humidity.max()
max_humidity

100

In [20]:
# Plot as a Terrain map
new_york_coordinates = (40.75, -74.00)

fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities with max temp between 70 and 85, wind speed less than 10 mph, and cloudiness equals to 0

ideal_vacation_cities = cities_df.loc[(cities_df["Temperature"] > 70) & (cities_df["Temperature"] < 85) & 
                        (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_vacation_cities = ideal_vacation_cities.dropna()

# Reset the index after dropping rows with null values
ideal_vacation_cities = ideal_vacation_cities.reset_index(drop=True)
ideal_vacation_cities

,Unnamed: 0,City,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Lat,Lng
0,2,east london,72.99,37,0,4.47,ZA,1592926883,-33.02,27.91
1,153,lumeje,71.74,25,0,2.95,AO,1592927331,-11.55,20.78
2,214,porto recanati,82.13,32,0,6.93,IT,1592927338,43.44,13.66
3,237,kanker,79.38,78,0,4.97,IN,1592927339,20.27,81.49
4,268,upington,75.20,17,0,8.05,ZA,1592927342,-28.45,21.26


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store data into new variable called hotel_df
hotel_df = ideal_vacation_cities

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set up a parameters dictionary
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}
# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # Find the lat, lng, and city name from each row in hotel_df
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    # Add location to params dict
    params["location"] = f"{lat}, {lng}"
    print(f"Retrieving Results for Index {index}: {city}.")
    
    # Assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response["results"]
    
    # Find the closest hotel to each city and store the name back in the hotel_df
    try:
        print("Closest hotel to " + city + " is " + results[0]["name"])
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    # Handle exceptions in case there is a KeyError or IndexError...
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Display the hotel_df
hotel_df
    
    



Retrieving Results for Index 0: east london.
Closest hotel to east london is Premier Hotels & Resorts
------------
Retrieving Results for Index 1: lumeje.
Missing field/result... skipping.
------------
Retrieving Results for Index 2: porto recanati.
Closest hotel to porto recanati is Hotel Il Brigantino
------------
Retrieving Results for Index 3: kanker.
Closest hotel to kanker is Hotel Green Palm
------------
Retrieving Results for Index 4: upington.
Closest hotel to upington is Protea Hotel by Marriott Upington
------------


,Unnamed: 0,City,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Lat,Lng,Hotel Name
0,2,east london,72.99,37,0,4.47,ZA,1592926883,-33.02,27.91,Premier Hotels & Resorts
1,153,lumeje,71.74,25,0,2.95,AO,1592927331,-11.55,20.78,
2,214,porto recanati,82.13,32,0,6.93,IT,1592927338,43.44,13.66,Hotel Il Brigantino
3,237,kanker,79.38,78,0,4.97,IN,1592927339,20.27,81.49,Hotel Green Palm
4,268,upington,75.20,17,0,8.05,ZA,1592927342,-28.45,21.26,Protea Hotel by Marriott Upington


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [8]:
hotel_locations

,Lat,Lng
0,-33.02,27.91
1,-11.55,20.78
2,43.44,13.66
3,20.27,81.49
4,-28.45,21.26


In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotel_locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))